## **Part A. ResNet Training Competition**
### **Import necessary libraries** 

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

### **Download CIFAR-100 dataset**



In [0]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.26733, 0.2564, 0.2762))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5089, 0.4874, 0.4419), (0.2683, 0.2574, 0.2771))
])

trainset = torchvision.datasets.CIFAR100(root='./data/cifar100', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(root='./data/cifar100', train=False, download=True, transform=transform_test)

trainloader= torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

### **Implement basic block & bottleneck block**

In [0]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [0]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

### **Build ResNet using predefined blocks**

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [0]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])

### **Implement train & test functions**

In [0]:
def train(model, trainloader, criterion, optimizer):    
    train_loss = 0.0
    train_acc = 0.0
    total = 0 
    correct = 0 
    
    train_loader = trainloader
    model.train()

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(inputs) 
        loss = criterion(output, labels)
        
        _, preds = output.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc = (correct / total) * 100.0 
        
    return (train_loss / (batch_idx + 1)) , train_acc

In [0]:
def test(model, testloader, criterion):
    test_loss = 0.0
    test_acc = 0.0
    total = 0
    correct = 0 
    
    test_loader = testloader
    model.eval()
    with torch.no_grad():
      for batch_idx, (inputs, labels) in enumerate(test_loader):
          inputs = inputs.to(device)
          labels = labels.to(device)
          
          optimizer.zero_grad()
          
          output = model(inputs)
          loss = criterion(output, labels)
          
          _, preds = output.max(1)
          total += labels.size(0)
          correct += preds.eq(labels).sum().item()
          
          test_loss += loss.item()
          test_acc = (correct / total) * 100.0
        
    return (test_loss / (batch_idx +1)), test_acc


### **Define the model and hyperparameters**

In [0]:
"""
TODO 
Hyperparameter tuning을 통하여 최대한 test accuracy를 끌어올리는 것이 목표입니다.
Rules:
1) 학생 분들께서 background가 제각각 다르기때문에 공정성을 위하여
모델은 주어진 5개의 resnet variation들만 사용할 수 있습니다. 
2) Pytorch에서 제공되는 pretrain된 resnet또한 사용하지 말아주세요. 
"""

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = 

num_epochs = 

model = model.to(device)
params_to_update = model.parameters()
optimizer = 
criterion = nn.CrossEntropyLoss()

### **Training loop**
### **이 셀의 아웃풋은 절대 지우시면 안됩니다. 아웃풋이 있는 상태로 노트북을 저장해서 submit 해주세요**
### **저장 된 모델 파일을 우클릭하시면 컴퓨터에 저장을 할 수 있습니다. 그 모델 또한 최종적으로 ipynb파일과 함께 압축하여 보내주셔야 합니다.**

In [0]:
for epoch in range(num_epochs):    
    train_loss, train_acc = train(model, trainloader, criterion, optimizer)
    if epoch % 10 == 0:
        test_loss, test_acc = test(model, testloader, criterion)
        print(f'Train accuracy: {train_acc} / Test accuracy: {test_acc}')

if os.path.isdir('./saved_models') == False:
    os.makedirs('./saved_models')

torch.save(model, './saved_models/resnet.pt')

## **Part B. Adversarial Attack**
### **Import necessary libraries**

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

### **Implement FGSM Attack**

In [0]:
def fgsm_attack(image, epsilon, data_grad):
  """
  TODO
  Clean image, epsilon = 0.01, gradient w.r.t. data 가 주어졌을 때,
  새로운 perturbed_image 를 생성해 주세요.
  이 perturbed_image는 다음 test_attack 함수에서 
  실제로 이 attack이 성공 하였는지 확인하기 위해 쓰이게 됩니다.
  받은 인풋을 사용하여 fgsm 구현을 하기 위한 식은 ppt 자료를 참고해주세요.
  """
    return perturbed_image


### **Implement Attacking Function**

In [0]:
def test_attack(model, device, test_loader, epsilon):
  """
  TODO
  1. test_loader의 데이터를 gpu로 옮겨줍니다.
  2. data에 대한 gradient를 구해주기 위해 data.requires_grad 를 True 로 만들어줍니다. (중요)
  3. 오리지널 (클린) 데이터로 forward pass를 해줍니다.
  4. crossentropyloss 값을 구하고, gradient를 구해줍니다.
  5. 계산 된 gradient 들 중 data gradient 만 뽑기 위해 data.grad.data 명령어를 사용하여 data_grad를 만들어줍니다.
  -> 여기까지 하시면 fgsm_attack 에 필요한 모든 인풋이 계산이 되게 됩니다. 
  6. fgsm_attack을 불러서 새로운 (어택이 된) 데이터를 생성해줍니다.
  7. 이 새로운 데이터를 model에 forward pass를 해줍니다.
  8. 새로운 예측값이 맞았는지 확인해줍니다.
  9. 예측값이 맞는 데이터의 갯수를 전체 데이터 갯수 값으로 나눠 줘서 마지막 test accuracy (final_acc)를 계산합니다.
  -> attack이 성공 적이었다면 final_acc가 전보다 현저히 낮아짐을 확인 할 수 있습니다.
  """
  
    print(f"Final Test Accuracy = {final_acc}")

    # Return the accuracy and an adversarial example
    return final_acc

### **Attack the model given epsilon = 0.01**
### **이 셀의 아웃풋은 절대 지우시면 안됩니다. 아웃풋이 있는 상태로 노트북을 저장해서 submit 해주세요**


In [0]:
epsilon = 0.01

model.eval()
test_attack(model, device, testloader, epsilon)